# Experimental: Advanced Noise Optimisation & Binary Encodings (QFT / Cuccaro)
**Purpose:** Archive experimental code for QFT-based binary encoding, Cuccaro-style incrementers, ancilla based incrementer and attempts to make a compact binary Galton engine. This is R&D material and is **experimental** — not part of the core deliverable due to fragility across Qiskit versions.

## Goals
1. Implement binary-encoded Galton engine variants using QFT-based increment and Cuccaro (MAJ/UMA).
2. Test deterministic mapping (exhaustive) for m=1..5 and collect failure cases.
3. Compare behavior, diagnose register-order & decomposition issues, and capture diagnostics for the final report.


In [ ]:
#Cell 1 : Installation
# Core Qiskit Terra + Aer
!pip install qiskit==1.3
!pip install qiskit-aer==0.15
!pip install qiskit-ibm-runtime==0.34.0

# Utilities
!pip install matplotlib numpy
!pip install pylatexenc==2.10

## Attempts and diagnostics summary
*Below are the method we tried*
- QFT-based increment: passed for m=1, failed for m>=2; failures typically manifested as high-order bit flips for certain inputs.
- Cuccaro ancilla-chain attempts: similar symptoms; causes traced to indexing & ancilla compute/uncompute sequencing.
- We include the failing outputs (example_fail lines) and `qc.draw()` snapshots in this notebook for reproducibility.

## Archive notes
- Keep the code and notebook in `experimental/` with a clear README explaining :
  - Qiskit versions used when experiments were run
  - Example failing mappings and raw counts
  - Recommended directions to continue (register-based CDKM or explicit MAJ/UMA using QuantumRegister objects)

## Why these are archived ?
- The QFT/Cuccaro/Ancilla variants are promising but depend on subtle register order, simulator/optimizer behavior and Qiskit API versions.
- Under tight time constraints we document and archive the experiments and focus deliverables on robust ones (one-hot & parallel samplers).


In [ ]:
# quick test for QFT-based binary Galton
from binary_galton_qft import verify_binary_qft, run_binary_galton_qft
# small noiseless verification
res = verify_binary_qft(k_list=(1,2,3,4), shots=4000, noise_model=None, verbose=True)
print("Verification results:", res)

# single-run example for k=8 (noiseless), compare to analytic
k = 8
counts = run_binary_galton_qft(k, shots=5000, seed=42, noise_model=None)
print("k=8 total counts:", sum(counts.values()))
from noise_optimisation import compare_metrics
expected = [__import__('math').comb(k,i)*(0.5**k)*5000 for i in range(k+1)]
metrics = compare_metrics(counts, expected, n_bins=k+1)
print("k=8 metrics (noiseless):", metrics)

In [ ]:
# Deterministic unit/mapping test for controlled increment (QFT variant)
from binary_galton_qft import build_binary_galton_qft_circuit, _controlled_increment_qft
from qiskit import QuantumCircuit, transpile
try:
    from qiskit_aer import AerSimulator
except Exception:
    from qiskit.providers.fake_provider import GenericBackendV2

def test_increment_mapping(m):
    """Test controlled increment mapping for register size m (no H on coin)."""
    reg = list(range(0, m))
    coin = m
    total = m + 1
    sim = AerSimulator()
    failures = []
    for r in range(2**m):
        # build a tiny circuit: set reg to r, set coin=1, apply controlled increment
        qc = QuantumCircuit(total, m)
        # prepare register in basis state r (LSB at reg[0] convention used in module)
        for i in range(m):
            if (r >> i) & 1:
                qc.x(reg[i])
        qc.x(coin)   # set coin = 1
        # apply the controlled increment gadget directly
        _controlled_increment_qft(qc, coin, reg)
        qc.measure(list(reversed(reg)), list(range(m)))
        tqc = transpile(qc, sim)
        res = sim.run(tqc, shots=2048).result()
        counts = res.get_counts()
        # determine most likely output integer
        # normalize keys to int
        out = max(counts.items(), key=lambda kv: kv[1])[0].replace(' ', '')
        out_i = int(out, 2)
        expected = (r + 1) % (2**m)
        if out_i != expected:
            failures.append((m, r, out_i, expected, counts))
    return failures

# Run for m = 1..4
for m in (1,2,3,4):
    fails = test_increment_mapping(m)
    print(f"m={m}  failures: {len(fails)}")
    if len(fails) > 0:
        print("Example failure (m, r, out, expected, counts):")
        print(fails[0])

In [ ]:
# Notebook cell: monkey-patch the controlled increment function (Variant 2)
import importlib
import binary_galton_qft as bgq

from qiskit import QuantumCircuit
from typing import List

def _controlled_increment_qft_fixed(qc: QuantumCircuit, coin: int, reg: List[int]) -> None:
    """
    Fixed variant: apply controlled phase rotations to reversed(reg) so that
    the LSB significance receives the smallest-angle rotation.
    """
    import math
    m = len(reg)
    if m == 0:
        return
    # Apply QFT on the register as before
    bgq.qft(qc, reg, do_swaps=False)
    # Apply CP to the reversed register ordering
    rev = list(reversed(reg))
    for j in range(m):
        angle = 2.0 * math.pi / (2 ** (j + 1))
        qc.cp(angle, coin, rev[j])
    # inverse QFT on original reg ordering
    bgq.iqft(qc, reg, do_swaps=False)

# Monkey-patch into module
bgq._controlled_increment_qft = _controlled_increment_qft_fixed

# Reload helpers (if you want), then run the deterministic mapping test
importlib.reload(bgq)

# Re-run the deterministic mapping test you used earlier:
from binary_galton_qft import build_binary_galton_qft_circuit, _controlled_increment_qft
def run_mapping_test_and_print():
    from binary_galton_qft import _controlled_increment_qft as inc
    from qiskit import transpile
    try:
        from qiskit_aer import AerSimulator
    except Exception:
        from qiskit.providers.aer import AerSimulator
    sim = AerSimulator()
    for m in (1,2,3,4):
        reg = list(range(0, m))
        coin = m
        total = m + 1
        fails = []
        for r in range(2**m):
            qc = QuantumCircuit(total, m)
            for i in range(m):
                if (r >> i) & 1:
                    qc.x(reg[i])
            qc.x(coin)
            inc(qc, coin, reg)
            qc.measure(list(reversed(reg)), list(range(m)))
            tqc = transpile(qc, sim)
            res = sim.run(tqc, shots=2048).result()
            counts = res.get_counts()
            out = max(counts.items(), key=lambda kv: kv[1])[0].replace(' ', '')
            out_i = int(out, 2)
            expected = (r + 1) % (2**m)
            if out_i != expected:
                fails.append((m, r, out_i, expected, counts))
        print(f"m={m} failures: {len(fails)}")
        if fails:
            print("Example failure:", fails[0])

run_mapping_test_and_print()

In [ ]:
import importlib
import binary_galton_qft as bgq
importlib.reload(bgq)
bgq.verify_binary_qft(k_list=(1,2,3,4), shots=4000, noise_model=None, verbose=True)

In [ ]:
import importlib
import binary_galton_qft as bgq
importlib.reload(bgq)
bgq.verify_binary_qft(k_list=(1,2,3,4), shots=4000, noise_model=None, verbose=True)

In [ ]:
# Diagnostic cell for binary_galton_qft_fixed
import importlib
import binary_galton_qft_fixed as bgq
importlib.reload(bgq)
from qiskit import transpile
try:
    from qiskit_aer import AerSimulator
except Exception:
    from qiskit_aer import AerSimulator
sim = AerSimulator()

def full_mapping_test_variant(m, variant_name):
    """Test the detected variant for all r in 0..2^m-1. Returns list of failures."""
    # find the function for variant_name
    vn = None
    for name, fn in bgq.CANDIDATE_VARIANTS:
        if name == variant_name:
            vn = fn
            break
    if vn is None:
        return [("no_variant_found", )]
    reg = list(range(m))
    coin = m
    total = m + 1
    fails = []
    for r in range(2**m):
        qc = bgq.QuantumCircuit(total, m) if hasattr(bgq, "QuantumCircuit") else None
        # build qc manually to ensure proper object:
        from qiskit import QuantumCircuit
        qc = QuantumCircuit(total, m)
        # prepare r:
        for i in range(m):
            if (r >> i) & 1:
                qc.x(reg[i])
        qc.x(coin)
        # apply candidate (vn is a lambda wrapper in list; call it)
        try:
            vn(qc, coin, reg)
        except Exception as e:
            fails.append(("exception", m, r, str(e)))
            continue
        qc.measure(list(reversed(reg)), list(range(m)))
        tqc = transpile(qc, sim)
        res = sim.run(tqc, shots=2048, seed_simulator=1234).result()
        counts = res.get_counts()
        out = max(counts.items(), key=lambda kv: kv[1])[0].replace(" ", "")
        out_i = int(out, 2)
        expected = (r + 1) % (2**m)
        if out_i != expected:
            fails.append((m, r, out_i, expected, counts))
    return fails

# Print detected variants and run full mapping
for m in (1,2,3,4,5):
    chosen = bgq.detect_working_variant_for_m(m)
    print(f"m={m} -> detected variant: {chosen}")
    if chosen is None:
        print("  No detected variant — will fail.")
        continue
    fails = full_mapping_test_variant(m, chosen)
    print(f"  full mapping failures: {len(fails)}")
    if len(fails) > 0:
        print("  example failure:", fails[0])

In [ ]:
# Improved detection: insist variant works for several basis inputs
import importlib
import binary_galton_qft_fixed as bgq
importlib.reload(bgq)
from qiskit import transpile
try:
    from qiskit_aer import AerSimulator
except Exception:
    from qiskit_aer import AerSimulator
sim = AerSimulator()

def _test_variant_m_multi(m: int, variant_fn, shots: int = 1024, samples=None) -> bool:
    if samples is None:
        # choose a few representative r values (0,1,2, max-1)
        samples = [0]
        if m >= 1:
            samples += [1]
        if m >= 2:
            samples += [2]
        samples += [max(0, 2**m - 1)]
        samples = sorted(set(samples))
    reg = list(range(m)); coin = m; total = m+1
    for r in samples:
        qc = QuantumCircuit(total, m)
        for i in range(m):
            if (r >> i) & 1:
                qc.x(reg[i])
        qc.x(coin)
        # build via variant_fn
        try:
            variant_fn(qc, coin, reg)
        except Exception:
            return False
        qc.measure(list(reversed(reg)), list(range(m)))
        tqc = transpile(qc, sim)
        res = sim.run(tqc, shots=shots, seed_simulator=1234).result()
        counts = res.get_counts()
        out = max(counts.items(), key=lambda kv: kv[1])[0].replace(" ", "")
        out_i = int(out, 2)
        if out_i != (r+1) % (2**m):
            return False
    return True

# Run multi-input detection for m=1..5 and report the first working variant
for m in (1,2,3,4,5):
    good = None
    for name, fn in bgq.CANDIDATE_VARIANTS:
        if _test_variant_m_multi(m, fn):
            good = name
            break
    print("m", m, "multi-input good variant:", good)

In [ ]:
# quick verify for Cuccaro/mcx-based increment
from binary_galton_cuccaro import _mapping_test_cuccaro, verify_cuccaro, run_binary_galton_cuccaro
# deterministic mapping test for m=1..5
fails = _mapping_test_cuccaro(max_m=5)
for m, fl in fails.items():
    print(f"m={m} failures: {len(fl)}")
    if fl:
        print("example fail:", fl[0])

# small-k verify
res = verify_cuccaro(k_list=(1,2,3,4), shots=4000, noise_model=None, verbose=True)
print("verify results:", res)

# single k=8 run (noiseless)
k = 8
counts = run_binary_galton_cuccaro(k, shots=5000, seed=42, noise_model=None)
print("k=8 counts total:", sum(counts.values()))
from noise_optimisation import compare_metrics
expected = [__import__('math').comb(k,i)*(0.5**k)*5000 for i in range(k+1)]
metrics = compare_metrics(counts, expected, n_bins=k+1)
print("k=8 metrics (noiseless):", metrics)

In [ ]:
# binary_galton_increment_ancilla (Advanced Method 2)
from binary_galton_increment_ancilla import mapping_test_increment, run_binary_galton_increment
fails = mapping_test_increment(max_m=5)
print({m: len(fails[m]) for m in sorted(fails)})
for m, fl in fails.items():
    if fl:
        print("example fail for m=", m, fl[0])
# quick verify of small k
for k in (1,2,3,4,8):
    counts = run_binary_galton_increment(k, shots=4000, seed=1234)
    print("k=", k, "total_counts=", sum(counts.values()), "counts=", counts)

In [ ]:
# Test : binary_galton_increment_ancilla (Advanced Method 2)

from qiskit import transpile
from qiskit_aer import AerSimulator
from binary_galton_increment_ancilla import controlled_increment_with_ancillas

from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator

m = 2   # example failing m
r = 0   # example failing r (register input)
# build the same layout as mapping test
reg = list(range(0, m))
coin = m
ancillas = list(range(m + 1, m + 1 + max(0, m - 1)))
total = m + 1 + max(0, m - 1)
qc = QuantumCircuit(total, m)
# prepare r
for i in range(m):
    if (r >> i) & 1:
        qc.x(reg[i])
qc.x(coin)
controlled_increment_with_ancillas(qc, coin, reg, ancillas)
qc.measure(list(reversed(reg)), list(range(m)))
print(qc.draw(output='text'))
sim = AerSimulator()
tqc = transpile(qc, sim)
res = sim.run(tqc, shots=2048, seed_simulator=1234).result()
print(res.get_counts())

In [ ]:
# Debug helper: shows raw counts and both integer interpretations
from qiskit import transpile
from qiskit_aer import AerSimulator
from binary_galton_increment_ancilla import controlled_increment_with_ancillas
from qiskit import QuantumCircuit

def debug_instance(m, r):
    reg = list(range(0, m))
    coin = m
    ancillas = list(range(m + 1, m + 1 + max(0, m - 1)))
    total = m + 1 + max(0, m - 1)
    qc = QuantumCircuit(total, m)
    # prepare r in reg (LSB-first)
    for i in range(m):
        if (r >> i) & 1:
            qc.x(reg[i])
    qc.x(coin)  # coin = 1
    controlled_increment_with_ancillas(qc, coin, reg, ancillas)
    # measure: we measure reg into classical bits 0..m-1, but be explicit about ordering
    qc.measure(list(reversed(reg)), list(range(m)))  # measure reg[m-1]->c0 ... reg[0]->c[m-1]
    print("Circuit:")
    print(qc.draw(output='text'))
    sim = AerSimulator()
    tqc = transpile(qc, sim)
    res = sim.run(tqc, shots=2048, seed_simulator=1234).result()
    counts = res.get_counts()
    print("Raw counts keys (Qiskit MSB..LSB):", counts)
    # For each bitstring, show two interpretations
    for bs, c in counts.items():
        s = str(bs).replace(" ", "")
        msb_first = int(s, 2)
        lsb_first = int(s[::-1], 2)
        print(f" bitstring='{s}'  count={c}  -> msb_first={msb_first}, lsb_first={lsb_first}")
    return counts

# Run it for the failing examples you reported:
print("m=2, r=0:")
debug_instance(2, 0)
print("\nm=3, r=0:")
debug_instance(3, 0)
print("\nm=4, r=0:")
debug_instance(4, 0)

In [ ]:
# binary_galton_increment_regs.py (Advanced Method 3)
import importlib
import binary_galton_increment_regs as igr
importlib.reload(igr)
from qiskit import transpile
try:
    from qiskit_aer import AerSimulator
except Exception:
    from qiskit.providers.fake_provider import GenericBackendV2
sim = AerSimulator()

def mapping_test_regs(max_m=5):
    failures = {}
    for m in range(1, max_m+1):
        fails = []
        # build circuit template objects (we'll prepare distinct circuits per r)
        for r in range(2**m):
            qc, A, COIN, ANC, CREG = igr.build_increment_circuit_regs(m)
            # prepare reg = r (LSB-first)
            for i in range(m):
                if (r >> i) & 1:
                    qc.x(A[i])
            # set coin=1
            qc.x(COIN[0])
            igr.controlled_increment_regs(qc, A, COIN, ANC)
            qc.measure(list(reversed(A)), list(range(m)))
            tqc = transpile(qc, sim)
            res = sim.run(tqc, shots=2048, seed_simulator=1234).result()
            counts = res.get_counts()
            out_bs, _ = max(counts.items(), key=lambda kv: kv[1])
            out_i = int(str(out_bs).replace(" ", ""), 2)
            expected = (r + 1) % (2**m)
            if out_i != expected:
                fails.append((m, r, out_i, expected, counts))
        failures[m] = fails
    return failures

fails = mapping_test_regs(max_m=5)
print({m: len(fails[m]) for m in sorted(fails)})
for m,f in fails.items():
    if f:
        print("example fail:", f[0])